<a href="https://colab.research.google.com/github/jberkow713/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/Jberkow_Assignment_RNN%20and%20LTSM%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

import requests
import pandas as pd

In [0]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [5]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


In [6]:
df_toc['text'][1]

'THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRAMATIS PERSONAE\r\n\r\n  MARK ANTONY,         Triumvirs\r\n  OCTAVIUS CAESAR,         "\r\n  M. AEMILIUS LEPIDUS,     "\r\n  SEXTUS POMPEIUS,         "\r\n  DOMITIUS ENOBARBUS, friend to Antony\r\n  VENTIDIUS,             "    "   "\r\n  EROS,                  "    "   "\r\n  SCARUS,                "    "   "\r\n  DERCETAS,              "    "   "\r\n  DEMETRIUS,             "    "   "\r\n  PHILO,                 "    "   "\r\n  MAECENAS,   friend to Caesar\r\n  AGRIPPA,       "    "   "\r\n  DOLABELLA,     "    "   "\r\n  PROCULEIUS,    "    "   "\r\n  THYREUS,       "    "   "\r\n  GALLUS,        "    "   "\r\n  MENAS,      friend to Pompey\r\n  MENECRATES,    "    "    "\r\n  VARRIUS,       "    "    "\r\n  TAURUS, Lieutenant-General to Caesar\r\n  CANIDIUS, Lieutenant-General to Antony\r\n  SILIUS, an Officer in Ventidius\'s army\r\n  EUPHRONIUS, an Ambassador from Antony to Caesar\r\n  ALEXAS,   attendant on Cleopatra\r\n  MARDIAN,    

In [0]:
data = df_toc

In [8]:
print(data.shape)
data.head()

(43, 4)


,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


In [9]:
data.text.values[0]

'ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nContents\r\n\r\nACT I\r\nScene I. Rossillon. A room in the Countess’s palace.\r\nScene II. Paris. A room in the King’s palace.\r\nScene III. Rossillon. A Room in the Palace.\r\n\r\n\r\nACT II\r\nScene I. Paris. A room in the King’s palace.\r\nScene II. Rossillon. A room in the Countess’s palace.\r\nScene III. Paris. The King’s palace.\r\nScene IV. Paris. The King’s palace.\r\nScene V. Another room in the same.\r\n\r\n\r\nACT III\r\nScene I. Florence. A room in the Duke’s palace.\r\nScene II. Rossillon. A room in the Countess’s palace.\r\nScene III. Florence. Before the Duke’s palace.\r\nScene IV. Rossillon. A room in the Countess’s palace.\r\nScene V. Without the walls of Florence.\r\nScene VI. Camp before Florence.\r\nScene VII. Florence. A room in the Widow’s house.\r\n\r\n\r\nACT IV\r\nScene I. Without the Florentine camp.\r\nScene II. Florence. A room in the Widow’s house.\r\nScene III. The Florentine camp.\r\nScene IV. Florence. A room in 

In [0]:
text = " ".join(data.text.values)

chars = list(set(text))

char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [12]:

len(chars)

106

In [0]:
data['text_clean'] = data['text'].replace({r'[^\x00-\x7f]+':''}, regex=True, inplace=True)

In [14]:
text = " ".join(data.text.values)

chars = list(set(text))

char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 
len(chars)

90

In [15]:
chars

['d',
 'e',
 'B',
 'Q',
 '(',
 'G',
 '\r',
 'P',
 'M',
 'K',
 'o',
 '5',
 'Y',
 '\t',
 'E',
 'l',
 '?',
 '1',
 ']',
 'x',
 '6',
 '-',
 ':',
 'X',
 '_',
 ';',
 '9',
 '$',
 'T',
 'H',
 '3',
 'U',
 'v',
 '4',
 'j',
 'W',
 'a',
 'i',
 '0',
 's',
 '}',
 '"',
 '@',
 'C',
 "'",
 'm',
 'Z',
 '*',
 'D',
 '2',
 '\\',
 '`',
 'q',
 'V',
 '|',
 'k',
 'N',
 'h',
 'f',
 '/',
 't',
 'I',
 'L',
 'S',
 'n',
 '%',
 'A',
 'b',
 'r',
 'w',
 'F',
 '8',
 '.',
 ')',
 'y',
 '[',
 'z',
 'g',
 ' ',
 '&',
 'u',
 'J',
 'c',
 '\n',
 '!',
 'R',
 ',',
 '7',
 'p',
 'O']

In [16]:
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1123491


In [17]:
len(encoded)

5617494

In [18]:
sequences[0]

[66,
 62,
 62,
 63,
 78,
 35,
 14,
 62,
 62,
 78,
 28,
 29,
 66,
 28,
 78,
 14,
 56,
 48,
 63,
 78,
 35,
 14,
 62,
 62,
 6,
 83,
 6,
 83,
 6,
 83,
 6,
 83,
 43,
 10,
 64,
 60,
 1,
 64,
 60,
 39]

In [0]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [20]:
# Shape is represented by : Sequences, max length char per sequence, characters
x.shape

(1123491, 40, 90)

In [21]:
# Shape is represented by : Sequences, characters

y.shape

(1123491, 90)

In [0]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')



In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(600):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [28]:
model.fit(x, y,
          batch_size=32,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
35100/35110 [============================>.] - ETA: 0s - loss: 1.3689
----- Generating text after Epoch: 0
----- Generating with seed: "and Kent_] you to your rights;
With boo"
and Kent_] you to your rights;
















When, tirnces sha
35110/35110 [==============================] - 179s 5ms/step - loss: 1.3689
Epoch 2/10
35102/35110 [============================>.] - ETA: 0s - loss: 1.3617
----- Generating text after Epoch: 1
----- Generating with seed: "

EMILIA.
Why, would not you?

DESD"


EMILIA.
Why, would not you?















And hear you to 4: Our souls shoul
35110/35110 [==============================] - 179s 5ms/step - loss: 1.3617
Epoch 3/10
35103/35110 [============================>.] - ETA: 0s - loss: 1.3564
----- Generating text after Epoch: 2
----- Generating with seed: ";
Commend them and condemn them to her "
;










    Of him made jest ome

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.






Why I say sinni
35110/35110 [==============================] - 176s 5ms/step - loss: 1.3564
Epoch 4/10
35107/35110 [============================>.] - ETA: 0s - loss: 1.3500
----- Generating text after Epoch: 3
----- Generating with seed: "any possible satisfaction, and so would "
















    Is think? I peace, you shall not sharp
35110/35110 [==============================] - 179s 5ms/step - loss: 1.3500
Epoch 5/10
35106/35110 [============================>.] - ETA: 0s - loss: 1.3449
----- Generating text after Epoch: 4
----- Generating with seed: "ther plays the rogue with my great toe. "



















    And the
35110/35110 [==============================] - 181s 5ms/step - loss: 1.3449
Epoch 6/10
35103/35110 [============================>.] - ETA: 0s - loss: 1.3411
----- Generating text after Epoch: 5
----- Generating with seed: "cross shall part;
            You and y"
cross shall part;














  CORIN. Look
35110/35110 [==============================] - 180

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN